# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Описание данных

**Признаки:**
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя  
    **Целевой признак:** 
- Price — цена (евро)

## План:

**Подготовка данных**
- Подключаем библиотеки и открываем датафрейм
- Исследуем пропуски.  
    - Приводим возможные причины подобных пропусков  
    - Заполняем пропуски   
- Приводим данные к нужным типам, стандартизируем колличественные признаки
- Находим и устраняем ошибки в данных
- Разделяем исходные данные на обучающую, валидационную и тестовую выборки

**Обучение моделей**
- Обучим разные модели. Для каждой подберем различные гиперпараметры.
    - Для оценки качества моделей используем метрику RMSE, как наиболее скоростную
    - Первую модель будем строить из библиотеки LightGBM и её средствами устроим градиентный бустинг.
- Проанализируем скорость работы и качество моделей.

## Подготовка данных

- Подключаем библиотеки и открываем датафрейм
- Исследуем пропуски.  
    - Приводим возможные причины подобных пропусков  
    - Заполняем пропуски   
- Приводим данные к нужным типам, стандартизируем колличественные признаки
- Находим и устраняем ошибки в данных
- Разделяем исходные данные на обучающую, валидационную и тестовую выборки

In [1]:
import pandas as pd
from scipy import stats as st
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score  
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
!pip install lightgbm==3.2.1
import lightgbm as lgb
from lightgbm import LGBMRegressor
!pip install catboost
from catboost import CatBoostRegressor
from sklearn.metrics import roc_auc_score
import time

from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

  Using cached lightgbm-3.2.1-py3-none-win_amd64.whl (1.0 MB)
  Using cached graphviz-0.19.1-py3-none-any.whl (46 kB)


In [35]:
# <чтение файла с данными с сохранением в data>

Wall time: 6.19 s


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


### Ищем пропуски по столбцах

In [8]:
data.isnull().sum() / len(data) * 100

DateCrawled           0.000000
Price                 0.000000
VehicleType          10.579368
RegistrationYear      0.000000
Gearbox               5.596709
Power                 0.000000
Model                 5.560588
Kilometer             0.000000
RegistrationMonth     0.000000
FuelType              9.282697
Brand                 0.000000
NotRepaired          20.079070
DateCreated           0.000000
NumberOfPictures      0.000000
PostalCode            0.000000
LastSeen              0.000000
dtype: float64

Итого у нас 5 столбцов с пропусками:  
- 10% в типе кузова `VehicleType` 
- 5% в типе коробки передач `Gearbox` 
- 5% случаях не указана модель `Model`
- 9% не указан тип топлива `FuelType`
- 20% не указано отсутвие или наличие ремонта `NotRepaired`  

Предлагаемые меры борьбы с пропусками:
- тип кузова, коробка передач, тип топлива зависит от модели и года выпуска
- на модель надо смотреть подробнее что за пропуски
- пропуски в столбце с ремонтом заменим на "unknown"  

Начем с пропусков с столбце с моделями

In [9]:
data[data['Model'].isnull()].head(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
59,2016-03-29 15:48:15,1,suv,1994,manual,286,NaN,150000,11,NaN,sonstige_autos,NaN,2016-03-29 00:00:00,0,53721,2016-04-06 01:44:38
81,2016-04-03 12:56:45,350,small,1997,manual,54,NaN,150000,3,NaN,fiat,yes,2016-04-03 00:00:00,0,45665,2016-04-05 11:47:13
115,2016-03-20 18:53:27,0,small,1999,NaN,0,NaN,5000,0,petrol,volkswagen,NaN,2016-03-20 00:00:00,0,37520,2016-04-07 02:45:22
135,2016-03-27 20:51:23,1450,sedan,1992,manual,136,NaN,150000,0,NaN,audi,no,2016-03-27 00:00:00,0,38709,2016-04-05 20:17:31
151,2016-03-27 20:47:22,6799,small,2009,NaN,60,NaN,20000,5,petrol,volkswagen,no,2016-03-27 00:00:00,0,89077,2016-03-27 20:47:22
160,2016-03-19 19:49:15,500,small,1999,manual,0,NaN,150000,0,petrol,renault,no,2016-03-19 00:00:00,0,26969,2016-04-01 11:17:48
161,2016-03-28 10:50:04,1495,wagon,2001,NaN,64,NaN,150000,9,gasoline,volkswagen,NaN,2016-03-28 00:00:00,0,99086,2016-04-04 11:45:46
186,2016-03-16 15:51:08,14000,sedan,2008,NaN,235,NaN,150000,0,NaN,bmw,no,2016-02-12 00:00:00,0,95131,2016-04-07 14:56:12
193,2016-03-27 12:47:45,1200,coupe,1990,manual,45,NaN,150000,0,petrol,volkswagen,NaN,2016-03-27 00:00:00,0,83317,2016-04-07 08:17:08


Никаких особых аномальных закономерностей не прослеживается, следовательно, виной данных пропусков является пользователи, которые по каким то причинам не заполнили данную графу. Поэтому поступим как с ремонтом, заменим на пропуски на "unknown", так как ни по марке автомобиля, ни как либо еще мы не можем уверенно предсказать модель.

In [36]:
def nan_count(column):
    print(f'Пропусков в столбце {column}:',data[column].isnull().sum())
def past_unknown(column):
    flag = (data[column].isna())
    data.loc[flag, column] = 'unknown'
    nan_count(column)

past_unknown('Model')
past_unknown('NotRepaired')


Пропусков в столбце Model: 0
Пропусков в столбце NotRepaired: 0


Оставшиеся пропуски зависят от модели и года, поэтому мы их заполним средними значениями по данным столбцам. Для этого мы методом groupby, соберем данные по моделям к нему применим метод transform, для возврата того же самого датафрейма, но уже с заполненными значениями, и этим алгоритмом заполним пропущенные значения методом fillna
- transform Заменяет все пропущенные цифровые значения в датафрейме на медиану, а все категориальные на самые распространённые. Если какие то значения не находятся, то вместо них заполним `unknown`

In [37]:
# добавь условие если нет искомых значений
def rmv(x):
    if np.issubdtype(x.dtype, np.number) == True:
        return x.median() 
    elif np.issubdtype(x.dtype, np.object) == True:
        return 'unknown'
    return x.mode().iloc[0]   

In [38]:
data = data.fillna(data.groupby(['Model', 'RegistrationYear']).transform(rmv))
data.isnull().sum() / len(data) * 100

DateCrawled          0.0
Price                0.0
VehicleType          0.0
RegistrationYear     0.0
Gearbox              0.0
Power                0.0
Model                0.0
Kilometer            0.0
RegistrationMonth    0.0
FuelType             0.0
Brand                0.0
NotRepaired          0.0
DateCreated          0.0
NumberOfPictures     0.0
PostalCode           0.0
LastSeen             0.0
dtype: float64

In [7]:
data.isnull().sum() / len(data) * 100

DateCrawled          0.0
Price                0.0
VehicleType          0.0
RegistrationYear     0.0
Gearbox              0.0
Power                0.0
Model                0.0
Kilometer            0.0
RegistrationMonth    0.0
FuelType             0.0
Brand                0.0
NotRepaired          0.0
DateCreated          0.0
NumberOfPictures     0.0
PostalCode           0.0
LastSeen             0.0
dtype: float64

Класс вышел несколько тяжеловатым(время выполнения 20 мин на локалке), зато пропуски заполненны наиболее качественно, а функция вышла универсальной и маштабируемой. Ну и все пропуски устранены, а насколько качественно посмотрим в следующих сериях)

### Ищем аномалии
- в колличественных признаках
- в категориальных

In [8]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Что мы обнаружили:
- нулевые цены
- нереалистичные года вроде 1000 и 9999
- нулевая мощность двигателя
- нулевой месяц регистрации
- как то подозрительно мало фотографий
- неправильно указанные индексы(но на них как то пофиг)  
Разберем:

In [10]:
data[data['Price'] == 0]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
40,2016-03-26 22:06:17,0,small,1990,manual,0,corsa,150000,1,petrol,opel,unknown,2016-03-26 00:00:00,0,56412,2016-03-27 17:43:34
111,2016-03-19 18:40:12,0,convertible,2017,manual,0,golf,5000,12,petrol,volkswagen,unknown,2016-03-19 00:00:00,0,21698,2016-04-01 08:47:05
115,2016-03-20 18:53:27,0,small,1999,manual,0,unknown,5000,0,petrol,volkswagen,unknown,2016-03-20 00:00:00,0,37520,2016-04-07 02:45:22
152,2016-03-11 18:55:53,0,bus,2004,manual,101,meriva,150000,10,lpg,opel,yes,2016-03-11 00:00:00,0,27432,2016-03-12 23:47:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354205,2016-03-09 15:56:30,0,small,2000,manual,65,corsa,150000,0,petrol,opel,yes,2016-03-09 00:00:00,0,23758,2016-03-30 11:16:08
354238,2016-03-20 14:55:07,0,small,2002,manual,60,fiesta,150000,3,petrol,ford,unknown,2016-03-20 00:00:00,0,33659,2016-04-06 18:45:23
354248,2016-03-24 13:48:05,0,small,1999,manual,53,swift,150000,3,petrol,suzuki,unknown,2016-03-24 00:00:00,0,42329,2016-04-07 05:17:24
354277,2016-03-10 22:55:50,0,small,1999,manual,37,arosa,150000,7,petrol,seat,yes,2016-03-10 00:00:00,0,22559,2016-03-12 23:46:32


Данные нули это явно выбивающиеся значение, поэтому мы их заменим на свойственные данным моделям и годам цены. Дальше мы напишем класс что будет обрабатывать аномальные значения. Если ничего не найдется, то оставим нуль, так как это наиболее подходящее числовое значение для данного случая, а на не числовые повлияют на тип столбца.

In [39]:
%%time
class replacing_anamal_values:
    def id_col(self, df, target, features0, features1, flag):
        self.df = df
        self.column = target
        self.features0 = df[features0]
        self.features1 = df[features1]
        self.flag_false = flag == False
        df.loc[flag, target] = df[flag].apply(rav.past_values, axis=1)  

    def past_values(self, stroka):
        feat0 = stroka[self.features0.name]
        feat1 = stroka[self.features1.name]
        flag_false = (self.flag_false) & (self.features0 == feat0) & (self.features1 == feat1)
        if flag_false.sum() == 0:
            flag_false = (self.flag_false) & (self.features0 == feat0)
            if flag_false.sum() == 0:
                return 0
        return self.df.loc[flag_false, self.column].value_counts().index[0] 
    
rav = replacing_anamal_values()
rav.id_col(data, 'Price', 'Model', 'RegistrationYear', data['Price'] == 0)


Wall time: 4min 14s


посмотрим что там с годами

In [12]:
data[(data['RegistrationYear'] < 1969)]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
15,2016-03-11 21:39:15,450,small,1910,manual,0,ka,5000,0,petrol,ford,unknown,2016-03-11 00:00:00,0,24148,2016-03-19 08:46:47
622,2016-03-16 16:55:09,490,sedan,1111,manual,0,unknown,5000,0,petrol,opel,unknown,2016-03-16 00:00:00,0,44628,2016-03-20 16:44:37
1794,2016-04-03 21:49:57,12800,coupe,1965,manual,90,other,100000,8,petrol,ford,no,2016-04-03 00:00:00,0,12049,2016-04-05 23:15:21
1928,2016-03-25 15:58:21,7000,suv,1945,manual,48,other,150000,2,petrol,volkswagen,no,2016-03-25 00:00:00,0,58135,2016-03-25 15:58:21
2262,2016-03-09 14:45:05,6500,convertible,1965,auto,360,unknown,150000,6,petrol,sonstige_autos,unknown,2016-03-09 00:00:00,0,67105,2016-04-06 08:46:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352709,2016-03-19 15:25:45,15990,coupe,1964,auto,250,other,5000,1,petrol,chevrolet,no,2016-03-19 00:00:00,0,60488,2016-03-30 16:45:11
353531,2016-03-16 21:56:55,6000,sedan,1937,manual,38,other,5000,0,petrol,mercedes_benz,unknown,2016-03-16 00:00:00,0,23936,2016-03-30 18:47:41
353961,2016-03-17 13:54:22,200,small,1910,manual,0,unknown,5000,0,petrol,sonstige_autos,unknown,2016-03-17 00:00:00,0,42289,2016-03-31 22:46:47
354037,2016-03-23 16:57:30,10500,convertible,1968,manual,54,other,20000,4,petrol,renault,no,2016-03-23 00:00:00,0,63755,2016-03-25 00:47:18


Тут один шлак с неизвестными марками. Их можно с чистой совестью удалить

In [40]:
data = data[((data['RegistrationYear'] < 1970) & (data['Model'] == 'unknown') | (data['Model'] == 'other')) == False]

In [14]:
data[(data['RegistrationYear'] < 1969)]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
15,2016-03-11 21:39:15,450,small,1910,manual,0,ka,5000,0,petrol,ford,unknown,2016-03-11 00:00:00,0,24148,2016-03-19 08:46:47
7289,2016-03-09 23:56:48,9950,bus,1966,manual,0,transporter,80000,6,petrol,volkswagen,no,2016-03-09 00:00:00,0,38324,2016-03-09 23:56:48
11367,2016-03-08 14:47:38,9555,bus,1963,manual,52,transporter,150000,6,petrol,volkswagen,no,2016-03-08 00:00:00,0,92421,2016-03-08 14:47:38
12087,2016-03-25 15:38:43,260,sedan,1960,manual,0,kaefer,10000,0,petrol,volkswagen,unknown,2016-03-25 00:00:00,0,94315,2016-04-06 19:16:05
12992,2016-04-03 19:38:21,500,small,1954,manual,54,corsa,150000,0,petrol,opel,no,2016-04-03 00:00:00,0,24148,2016-04-03 19:38:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346068,2016-04-03 21:25:23,11999,sedan,1966,manual,150,s_klasse,20000,3,petrol,mercedes_benz,no,2016-04-03 00:00:00,0,35759,2016-04-05 12:10:39
349028,2016-03-31 23:49:38,900,sedan,1960,manual,0,golf,150000,0,petrol,volkswagen,yes,2016-03-31 00:00:00,0,17491,2016-04-07 01:44:37
349688,2016-03-13 17:46:22,1,sedan,1960,manual,0,601,5000,0,petrol,trabant,unknown,2016-03-13 00:00:00,0,26160,2016-03-20 14:47:05
351312,2016-03-27 12:48:38,9000,convertible,1967,manual,160,911,5000,12,petrol,porsche,no,2016-03-27 00:00:00,0,44575,2016-04-07 08:17:43


остались только вменяемые автомобили. От шлака избавились, давайте посмотрим что там с автомобилями будущего

In [15]:
data[data['RegistrationYear'] < 2021]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,sedan,1993,manual,0,golf,150000,0,petrol,volkswagen,unknown,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,2500,small,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,wagon,2005,manual,0,unknown,20000,1,petrol,sonstige_autos,unknown,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


Посмотрим что там с нулевой мощьностью двигателя Power

In [16]:
data[data['Power']==0]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,sedan,1993,manual,0,golf,150000,0,petrol,volkswagen,unknown,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
15,2016-03-11 21:39:15,450,small,1910,manual,0,ka,5000,0,petrol,ford,unknown,2016-03-11 00:00:00,0,24148,2016-03-19 08:46:47
32,2016-03-15 20:59:01,245,sedan,1994,manual,0,golf,150000,2,petrol,volkswagen,no,2016-03-15 00:00:00,0,44145,2016-03-17 18:17:43
37,2016-03-28 17:50:15,1500,bus,2016,manual,0,kangoo,150000,1,gasoline,renault,no,2016-03-28 00:00:00,0,46483,2016-03-30 09:18:02
40,2016-03-26 22:06:17,350,small,1990,manual,0,corsa,150000,1,petrol,opel,unknown,2016-03-26 00:00:00,0,56412,2016-03-27 17:43:34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354338,2016-03-31 19:52:33,180,small,1995,manual,0,unknown,125000,3,petrol,opel,unknown,2016-03-31 00:00:00,0,41470,2016-04-06 14:18:04
354346,2016-03-07 17:06:35,2600,wagon,2005,auto,0,c_klasse,150000,9,gasoline,mercedes_benz,unknown,2016-03-07 00:00:00,0,61169,2016-03-08 21:28:38
354363,2016-03-27 20:36:20,1150,bus,2000,manual,0,zafira,150000,3,petrol,opel,no,2016-03-27 00:00:00,0,26624,2016-03-29 10:17:23
354364,2016-03-21 09:50:58,2500,small,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49


В основном нормальные года и модели. Меняем ее нашим классом

In [41]:
rav.id_col(data, 'Power', 'Model', 'RegistrationYear', data['Power'] == 0)

In [18]:
data['RegistrationMonth'].value_counts()

0     34765
3     31797
6     29241
5     27146
4     27114
7     25208
10    24368
11    22772
12    22717
9     22310
1     21373
8     20977
2     19674
Name: RegistrationMonth, dtype: int64

34 тысячи наименований с нулевым месяцем в сочетании с имеющимся 12. Видимо это просто не указанный месяц регестрации. На эффективность модели обучения он не влияет, поэтому оставим как есть

In [19]:
data['NumberOfPictures'].value_counts()

0    329462
Name: NumberOfPictures, dtype: int64

количество в каждой строке нулевое. Абсолютно бесполезный столбец, настолько что его можно удалить. 

Теперь разберем категориальные признаки:

In [20]:
data.select_dtypes(include=['object', 'datetime']).describe()

,DateCrawled,VehicleType,Gearbox,Model,FuelType,Brand,NotRepaired,DateCreated,LastSeen
count,329462,329462,329462,329462,329462,329462,329462,329462,329462
unique,257162,8,2,250,7,40,3,108,170881
top,2016-03-24 14:49:47,sedan,manual,golf,petrol,volkswagen,no,2016-04-03 00:00:00,2016-04-07 00:45:17
freq,7,97693,266601,29232,226694,76229,229826,12756,16


С категориальными переменными все в порядке)

### Приводим данные к нужным типам, стандартизируем колличественные признаки

In [26]:
data.dtypes

DateCrawled          object
Price                 int64
VehicleType          object
RegistrationYear      int64
Gearbox              object
Power                 int32
Model                object
Kilometer             int64
RegistrationMonth     int64
FuelType             object
Brand                object
NotRepaired          object
DateCreated          object
PostalCode            int64
LastSeen             object
dtype: object

Все категорийные признаки переводим в категорийный формат, а также создаем отдельный датафрейм без дат

In [42]:
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
#data['Power'] = data['Power'].astype('int')
data.loc[:, cat_features] = data.loc[:, cat_features].astype('category')
df = data.drop(['DateCrawled','DateCreated','LastSeen', 'NumberOfPictures'], axis=1)


Price                   int64
VehicleType          category
RegistrationYear        int64
Gearbox              category
Power                   int64
Model                category
Kilometer               int64
RegistrationMonth       int64
FuelType             category
Brand                category
NotRepaired          category
PostalCode              int64
dtype: object

In [43]:
df.dtypes.value_counts()

int64       6
category    1
category    1
category    1
category    1
category    1
category    1
dtype: int64

### Разделим данные на обучающую, валидационную и тестовую выборки

In [44]:
df_model, df_test = train_test_split(df, test_size=0.20, random_state=12347) # сразу выделим 20% данных для тестовой выборки
features_test = df_test.drop(['Price'], axis=1)
target_test = df_test['Price']

features = df_model.drop(['Price'], axis=1)
target = df_model['Price']
# и чтобы получить валидационную выборку в 20% от изначальной, мы берем 25% от оставшихся 80%
features_train, features_valid, target_train, target_valid = train_test_split(
features, target, test_size=0.25, random_state=12347)

## Обучение моделей
- Обучим разные модели. Для каждой подберем различные гиперпараметры. Предполагаю планку по 
    - Для оценки качества моделей используем метрику RMSE, как наиболее скоростную
    - CatBoostRegressor
    - Модель из библиотеки LightGBM.
- Проанализируем скорость работы и качество моделей.

In [68]:
def modeling(model):
    cat_features = ['VehicleType', 'Gearbox', 'Model',
                'FuelType', 'Brand', 'NotRepaired']
    model = model
    start_fit = time.time()
    
    model.fit(features_train, target_train, cat_features=cat_features, verbose=50)
    time_fit = time.time() - start_fit
    start_predict = time.time()
    predict  = model.predict(features_valid)
    time_predict = time.time() - start_predict
    print(f'Время обучения модели {time_fit}, время предсказания {time_predict}')
    return mean_squared_error(target_valid, predict)**0.5


### CatBoostRegressor
Находим:
- лучшую глубину модели CatBoostRegressor
- оптимальное количество деревьве
- лучшую скорость обучения

In [69]:
best_model = None
best_result = 0
for depth in range(6, 15):
    result = modeling(CatBoostRegressor(loss_function="RMSE", iterations=150, random_seed = 12345, learning_rate =0.3, 
                              max_depth=depth))
    print(result, depth)

0:	learn: 3662.8288535	total: 97.3ms	remaining: 14.5s
50:	learn: 1733.4814081	total: 4.91s	remaining: 9.54s


KeyboardInterrupt: 

Теперь подберем скорость обучения.

In [49]:
best_model = None
best_result = 10000
rate = 0.3
while rate < 0.4:
    result = modeling(CatBoostRegressor(loss_function="RMSE", iterations=150, random_seed = 12345, learning_rate =rate,
                              max_depth=12))
    if result < best_result:
        best_result = result
        best_model = rate
    rate += 0.01
    
print(best_result, best_model)

0:	learn: 3542.8835042	total: 240ms	remaining: 35.7s
50:	learn: 1491.9025696	total: 11.6s	remaining: 22.5s
100:	learn: 1361.5325233	total: 23.1s	remaining: 11.2s
149:	learn: 1274.9189188	total: 34.9s	remaining: 0us
Время обучения модели 35.32914161682129, время предсказания 0.09873819351196289
0:	learn: 3513.7839170	total: 247ms	remaining: 36.8s
50:	learn: 1481.0389998	total: 12.5s	remaining: 24.3s
100:	learn: 1345.3107595	total: 25.5s	remaining: 12.4s
149:	learn: 1259.4744222	total: 37.6s	remaining: 0us
Время обучения модели 38.05832481384277, время предсказания 0.07281899452209473
0:	learn: 3484.8617136	total: 241ms	remaining: 35.9s
50:	learn: 1462.7078627	total: 12.6s	remaining: 24.4s
100:	learn: 1332.3558627	total: 24.3s	remaining: 11.8s
149:	learn: 1248.7511686	total: 36.1s	remaining: 0us
Время обучения модели 36.54386878013611, время предсказания 0.07177186012268066
0:	learn: 3456.1212617	total: 232ms	remaining: 34.6s
50:	learn: 1462.5370596	total: 12.3s	remaining: 23.9s
100:	lea

#### Оптимальные параметры CatBoostRegressor
Лучшая глубина модели : 12
Лучшая скорость обучения 0.3
Оптимальное количество интераций 150

### LGBMRegressor

In [55]:
def modeling_LGBMR(model):

    model = model
    start_fit = time.time()
    
    model.fit(features_train, target_train, categorical_feature='auto', verbose=50)
    time_fit = time.time() - start_fit
    start_predict = time.time()
    predict  = model.predict(features_valid)
    time_predict = time.time() - start_predict
    print(f'Время обучения модели {time_fit}, время предсказания {time_predict}')
    return mean_squared_error(target_valid, predict)**0.5


In [57]:
for depth in range(10, 61, 5):
    result = modeling_LGBMR(LGBMRegressor(boosting_type='gbdt', objective='regression', random_state = 12345, num_leaves=250,
                                learning_rate=0.17, n_estimators=150, max_depth=depth,
                                metric='rmse', bagging_fraction=0.8, feature_fraction=0.8, reg_lambda=0.9))
    print(result, depth)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
Время обучения модели 2.3299331665039062, время предсказания 0.3969385623931885
1521.2465207062799 10
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
Время обучения модели 2.2429111003875732, время предсказания 0.33823299407958984
1514.4722062293442 15
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
Время обучения модели 2.562145709991455, время предсказания 0.2932159900

In [59]:
for rate in range(150, 350, 25):
    result = modeling_LGBMR(LGBMRegressor(boosting_type='gbdt', objective='regression', random_state = 12345, num_leaves=250,
                                learning_rate=0.23, n_estimators=rate, max_depth=40,
                                metric='rmse', bagging_fraction=0.8, feature_fraction=0.8, reg_lambda=0.9))
    print(result, rate)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
Время обучения модели 2.2526979446411133, время предсказания 0.2503318786621094
1530.0362397272108 150
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
Время обучения модели 2.8576717376708984, время предсказания 0.32970428466796875
1529.296057516558 175
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
Время обучения модели 2.7806618213653564, время предсказания 0.36109972

In [61]:
best_model = None
best_result = 10000
rate = 0.1
while rate < 0.4:
    result = modeling_LGBMR(LGBMRegressor(boosting_type='gbdt', objective='regression', random_state = 12345, num_leaves=300,
                                learning_rate=rate, n_estimators=300, max_depth=50,
                                metric='rmse', bagging_fraction=0.8, feature_fraction=0.8, reg_lambda=0.9))
    if result < best_result:
        best_result = result
        best_model = rate
    rate += 0.01
    
print(best_result, best_model)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
Время обучения модели 5.7409563064575195, время предсказания 0.6703290939331055
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
Время обучения модели 5.325104475021362, время предсказания 0.6053807735443115
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
Время обучения модели 5.074746370315552, время предсказания 0.6243298053741455
[LightGBM] [Warning] feature_fraction i

#### Оптимальные параметры LGBMRegressor
- Лучшая глубина модели : 50 
- Лучшая скорость обучения 0.11
- Оптимальное количество интераций 300

## Анализ моделей

In [70]:
def modeling_analiz_cat(model):
    model = model
    start_fit = time.time()
    model.fit(features_train, target_train, cat_features=cat_features, verbose=50)
    time_fit = time.time() - start_fit
    start_predict = time.time()
    predict  = model.predict(features_valid)
    time_predict = time.time() - start_predict
    return mean_squared_error(target_valid, predict)**0.5, time_fit, time_predict
def modeling_analiz_LGBMR(model):
    model = model
    start_fit = time.time()
    model.fit(features_train, target_train, categorical_feature='auto', verbose=50)
    time_fit = time.time() - start_fit
    start_predict = time.time()
    predict  = model.predict(features_valid)
    time_predict = time.time() - start_predict
    return mean_squared_error(target_valid, predict)**0.5, time_fit, time_predict


In [72]:
rmse_cat, time_fit, time_predict = modeling_analiz_cat(CatBoostRegressor(loss_function="RMSE", iterations=150, random_seed = 12345,
                                                              learning_rate =0.3, max_depth=12))
cat = pd.DataFrame(data={
    'RMSE':[rmse_cat],
    'time_fit':[time_fit],
    'time_predict':[time_predict]},
                  index=['CatBoostRegressor'])

rmse_LGBMR, time_fit, time_predict = modeling_analiz_LGBMR(LGBMRegressor(boosting_type='gbdt', objective='regression',
                                                            random_state = 12345, num_leaves=225,
                                                            learning_rate=0.13, n_estimators=300, max_depth=50,
                                                            metric='rmse', bagging_fraction=0.8, feature_fraction=0.8,
                                                            reg_lambda=0.9))

LGBMR = pd.DataFrame(data={
    'RMSE':[rmse_LGBMR],
    'time_fit':[time_fit],
    'time_predict':[time_predict]},
                  index=['LGBMRegressor'])
itog = cat.append(LGBMR)

itog

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 3542.8835042	total: 236ms	remaining: 35.2s
50:	learn: 1491.9025696	total: 11.8s	remaining: 22.9s
100:	learn: 1361.5325233	total: 23.1s	remaining: 11.2s
149:	learn: 1274.9189188	total: 34.2s	remaining: 0us
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


,RMSE,time_fit,time_predict
CatBoostRegressor,1567.792408,34.631678,0.075798
LGBMRegressor,1505.075788,4.221646,0.522603


## Итоги:
- Мы выделяем 2 лучших модели, каждая из которых имеет свои особенности
- LGBMRegressor имеет наибольшую точность и среднюю скорость предсказаний
- CatBoostRegressor имеет чуть меньшую точность предсказания, в 5 раз больше времени чем LGBMRegressor скорость обучения, но почти мгновенную скорость предсказаний
- Старая добрая модель случайного леса, конечно может приблизиться по точности и полноте предсказаний к таким топовым бустерам как LGBMRegressor или CatBoostRegressor, но процесс обучения данной модели занимает в 10 раз больше времени, чем модель CatBoostRegressor
